In [13]:
import numpy
import pandas
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.regularizers import l2


In [14]:
train_data_dir = 'D:/My Documents/GIT Projects/IEEE-Hack-the-meta/ASL_F/asl_alphabet_train'
test_data_dir = 'D:/My Documents/GIT Projects/IEEE-Hack-the-meta/ASL_F/asl_alphabet_test'

In [15]:
train_datagen = ImageDataGenerator(
    rescale=1./255, 
    width_shift_range=0.3,
    height_shift_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(rescale=1./255)

In [16]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(150, 150),
    batch_size=64, 
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(150, 150),
    batch_size=64,
    class_mode='categorical'
)

Found 82650 images belonging to 29 classes.
Found 4411 images belonging to 29 classes.


In [17]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(29))
model.add(Activation('softmax'))

In [18]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',optimizer=optimizer, metrics=['accuracy'])

In [19]:
steps_per_epoch = len(train_generator)
validation_steps = len(test_generator)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=20,
    validation_data=test_generator,
    validation_steps=validation_steps,
    callbacks=[early_stopping],
)

Epoch 1/20
1292/1292 [==============================] - 393s 303ms/step - loss: 3.1456 - accuracy: 0.0904 - val_loss: 2.4478 - val_accuracy: 0.2847
Epoch 2/20
1292/1292 [==============================] - 390s 302ms/step - loss: 2.5859 - accuracy: 0.2145 - val_loss: 1.6793 - val_accuracy: 0.5065
Epoch 3/20
1292/1292 [==============================] - 463s 358ms/step - loss: 2.2041 - accuracy: 0.3009 - val_loss: 1.3858 - val_accuracy: 0.5568
Epoch 4/20
1292/1292 [==============================] - 488s 377ms/step - loss: 1.9447 - accuracy: 0.3666 - val_loss: 1.0890 - val_accuracy: 0.6522
Epoch 5/20
1292/1292 [==============================] - 469s 363ms/step - loss: 1.7568 - accuracy: 0.4202 - val_loss: 0.9907 - val_accuracy: 0.6686
Epoch 6/20
1292/1292 [==============================] - 467s 361ms/step - loss: 1.6170 - accuracy: 0.4577 - val_loss: 0.8151 - val_accuracy: 0.7200
Epoch 7/20
1292/1292 [==============================] - 675s 523ms/step - loss: 1.5106 - accuracy: 0.4887 - val_

In [20]:
score = model.evaluate(test_generator, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.27153244614601135
Test accuracy: 0.9152119755744934


In [9]:
model.save('asl_model.h5')